## Run ensemble from v1 on full dataset format commonly used on Kaggle

In [1]:
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from collections import Counter

# local imports
from prepare import *
from evaluate import *

In [2]:
def qwk3(a1, a2, max_rat=3):
    assert(len(a1) == len(a2))
    a1 = np.asarray(a1, dtype=int)
    a2 = np.asarray(a2, dtype=int)

    hist1 = np.zeros((max_rat + 1, ))
    hist2 = np.zeros((max_rat + 1, ))

    o = 0
    for k in range(a1.shape[0]):
        i, j = a1[k], a2[k]
        hist1[i] += 1
        hist2[j] += 1
        o +=  (i - j) * (i - j)

    e = 0
    for i in range(max_rat + 1):
        for j in range(max_rat + 1):
            e += hist1[i] * hist2[j] * (i - j) * (i - j)

    e = e / a1.shape[0]

    return 1 - o / e

### Read in initial datasets if needed

In [3]:
#raw_train, raw_train_labels, raw_test, specs, sample = read_raw_csvs()
#raw_train_labels = pd.read_csv('data/train_labels.csv')

### Load large train/test features from Josh's work

In [4]:
reduced_train = pd.read_csv('reduce_train.csv')
reduced_test = pd.read_csv('reduce_test.csv')
reduced_train.shape, reduced_test.shape

((17690, 891), (1000, 891))

In [5]:
#train = balance_classes(train)  ## try with balanced classes     --- results much worse

### Start throwing model mud at the wall

In [6]:
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, \
                                ExtraTreesClassifier, BaggingClassifier, \
                                GradientBoostingClassifier, VotingClassifier, StackingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
import warnings
warnings.filterwarnings('ignore')  #Ridge classifier throws some warnings about ill-conditioned matrix

### Baseline accuracy of 50% (or 25% if balanced)

In [7]:
reduced_train.accuracy_group.value_counts(normalize=True)

3    0.500000
0    0.239062
1    0.136292
2    0.124647
Name: accuracy_group, dtype: float64

# Initialize Models and start testing accuracy

#### KNN and SVC will require scaling, others shouldn't

In [8]:
rf = RandomForestClassifier()
#lr = LogisticRegression()               ## if they are commented out, they weren't performing well (or operator error...)
#sgd = SGDClassifier()
rc = RidgeClassifier()
#nb = GaussianNB()
ac = AdaBoostClassifier()
et = ExtraTreesClassifier()
bc = BaggingClassifier()
gbc = GradientBoostingClassifier()
clf = CatBoostClassifier(
    loss_function='MultiClass',
    task_type="CPU",
    learning_rate=0.01,
    iterations=2000,
    od_type="Iter",
    early_stopping_rounds=500,
    random_seed=42
    )

In [9]:
knn = KNeighborsClassifier()
svc = SVC(probability=True, verbose=1)

## Evaluate model performance

In [10]:
quick_eval(reduced_train, rf)

The accuracy of RandomForestClassifier is 0.5508762012436405
The QWK of RandomForestClassifier is 0.3703937710916171
              precision    recall  f1-score   support

           0       0.54      0.51      0.52       826
           1       0.28      0.11      0.15       509
           2       0.18      0.06      0.09       437
           3       0.60      0.82      0.69      1766

    accuracy                           0.55      3538
   macro avg       0.40      0.37      0.36      3538
weighted avg       0.49      0.55      0.50      3538



RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [11]:
# Catboost - accuracy 56%a
catbooster = quick_eval(reduced_train, clf, cv=True)

0:	learn: 1.3789760	total: 158ms	remaining: 5m 15s
1:	learn: 1.3721769	total: 250ms	remaining: 4m 10s
2:	learn: 1.3653490	total: 348ms	remaining: 3m 51s
3:	learn: 1.3586184	total: 460ms	remaining: 3m 49s
4:	learn: 1.3523827	total: 556ms	remaining: 3m 41s
5:	learn: 1.3462306	total: 651ms	remaining: 3m 36s
6:	learn: 1.3402063	total: 759ms	remaining: 3m 36s
7:	learn: 1.3345847	total: 867ms	remaining: 3m 35s
8:	learn: 1.3286372	total: 991ms	remaining: 3m 39s
9:	learn: 1.3232815	total: 1.12s	remaining: 3m 43s
10:	learn: 1.3178020	total: 1.24s	remaining: 3m 44s
11:	learn: 1.3125151	total: 1.34s	remaining: 3m 42s
12:	learn: 1.3071519	total: 1.44s	remaining: 3m 40s
13:	learn: 1.3021157	total: 1.53s	remaining: 3m 37s
14:	learn: 1.2971166	total: 1.64s	remaining: 3m 36s
15:	learn: 1.2920667	total: 1.74s	remaining: 3m 35s
16:	learn: 1.2873951	total: 1.84s	remaining: 3m 34s
17:	learn: 1.2827922	total: 1.94s	remaining: 3m 34s
18:	learn: 1.2784025	total: 2.05s	remaining: 3m 33s
19:	learn: 1.2741087	t

158:	learn: 1.0542785	total: 17.1s	remaining: 3m 18s
159:	learn: 1.0538786	total: 17.2s	remaining: 3m 18s
160:	learn: 1.0533977	total: 17.3s	remaining: 3m 18s
161:	learn: 1.0529702	total: 17.5s	remaining: 3m 18s
162:	learn: 1.0525839	total: 17.6s	remaining: 3m 18s
163:	learn: 1.0521497	total: 17.7s	remaining: 3m 18s
164:	learn: 1.0516652	total: 17.8s	remaining: 3m 17s
165:	learn: 1.0511167	total: 17.9s	remaining: 3m 17s
166:	learn: 1.0505464	total: 18s	remaining: 3m 17s
167:	learn: 1.0500819	total: 18.1s	remaining: 3m 17s
168:	learn: 1.0495961	total: 18.2s	remaining: 3m 17s
169:	learn: 1.0491341	total: 18.4s	remaining: 3m 17s
170:	learn: 1.0486838	total: 18.5s	remaining: 3m 17s
171:	learn: 1.0482236	total: 18.6s	remaining: 3m 18s
172:	learn: 1.0476574	total: 18.7s	remaining: 3m 17s
173:	learn: 1.0473671	total: 18.9s	remaining: 3m 17s
174:	learn: 1.0468848	total: 19s	remaining: 3m 17s
175:	learn: 1.0465825	total: 19.1s	remaining: 3m 17s
176:	learn: 1.0463022	total: 19.2s	remaining: 3m 1

316:	learn: 1.0114634	total: 34.5s	remaining: 3m 3s
317:	learn: 1.0113044	total: 34.6s	remaining: 3m 3s
318:	learn: 1.0110876	total: 34.7s	remaining: 3m 3s
319:	learn: 1.0109217	total: 34.8s	remaining: 3m 2s
320:	learn: 1.0106935	total: 34.9s	remaining: 3m 2s
321:	learn: 1.0105822	total: 35.1s	remaining: 3m 2s
322:	learn: 1.0104520	total: 35.2s	remaining: 3m 2s
323:	learn: 1.0102891	total: 35.3s	remaining: 3m 2s
324:	learn: 1.0101846	total: 35.4s	remaining: 3m 2s
325:	learn: 1.0100800	total: 35.5s	remaining: 3m 2s
326:	learn: 1.0099381	total: 35.6s	remaining: 3m 1s
327:	learn: 1.0098016	total: 35.7s	remaining: 3m 1s
328:	learn: 1.0096293	total: 35.8s	remaining: 3m 1s
329:	learn: 1.0095519	total: 35.9s	remaining: 3m 1s
330:	learn: 1.0093628	total: 36s	remaining: 3m 1s
331:	learn: 1.0092107	total: 36.1s	remaining: 3m 1s
332:	learn: 1.0090757	total: 36.3s	remaining: 3m 1s
333:	learn: 1.0089361	total: 36.4s	remaining: 3m 1s
334:	learn: 1.0087106	total: 36.5s	remaining: 3m 1s
335:	learn: 1.

474:	learn: 0.9898290	total: 52.9s	remaining: 2m 49s
475:	learn: 0.9897649	total: 53s	remaining: 2m 49s
476:	learn: 0.9897101	total: 53.1s	remaining: 2m 49s
477:	learn: 0.9895860	total: 53.2s	remaining: 2m 49s
478:	learn: 0.9894730	total: 53.4s	remaining: 2m 49s
479:	learn: 0.9894538	total: 53.5s	remaining: 2m 49s
480:	learn: 0.9893737	total: 53.6s	remaining: 2m 49s
481:	learn: 0.9892662	total: 53.8s	remaining: 2m 49s
482:	learn: 0.9891803	total: 53.9s	remaining: 2m 49s
483:	learn: 0.9890556	total: 54s	remaining: 2m 49s
484:	learn: 0.9889458	total: 54.2s	remaining: 2m 49s
485:	learn: 0.9887949	total: 54.3s	remaining: 2m 49s
486:	learn: 0.9886821	total: 54.4s	remaining: 2m 49s
487:	learn: 0.9884861	total: 54.5s	remaining: 2m 48s
488:	learn: 0.9883454	total: 54.6s	remaining: 2m 48s
489:	learn: 0.9882378	total: 54.8s	remaining: 2m 48s
490:	learn: 0.9882112	total: 54.9s	remaining: 2m 48s
491:	learn: 0.9881198	total: 55s	remaining: 2m 48s
492:	learn: 0.9880747	total: 55.2s	remaining: 2m 48s

631:	learn: 0.9744480	total: 1m 10s	remaining: 2m 32s
632:	learn: 0.9744018	total: 1m 10s	remaining: 2m 32s
633:	learn: 0.9743357	total: 1m 10s	remaining: 2m 32s
634:	learn: 0.9742796	total: 1m 10s	remaining: 2m 32s
635:	learn: 0.9742144	total: 1m 11s	remaining: 2m 32s
636:	learn: 0.9740709	total: 1m 11s	remaining: 2m 32s
637:	learn: 0.9740042	total: 1m 11s	remaining: 2m 32s
638:	learn: 0.9739825	total: 1m 11s	remaining: 2m 31s
639:	learn: 0.9738584	total: 1m 11s	remaining: 2m 31s
640:	learn: 0.9737439	total: 1m 11s	remaining: 2m 31s
641:	learn: 0.9736884	total: 1m 11s	remaining: 2m 31s
642:	learn: 0.9736630	total: 1m 11s	remaining: 2m 31s
643:	learn: 0.9735458	total: 1m 11s	remaining: 2m 31s
644:	learn: 0.9735176	total: 1m 11s	remaining: 2m 31s
645:	learn: 0.9734091	total: 1m 12s	remaining: 2m 31s
646:	learn: 0.9733115	total: 1m 12s	remaining: 2m 31s
647:	learn: 0.9732105	total: 1m 12s	remaining: 2m 30s
648:	learn: 0.9730449	total: 1m 12s	remaining: 2m 30s
649:	learn: 0.9729760	total:

...

1269:	learn: 0.9248197	total: 2m 17s	remaining: 1m 18s
1270:	learn: 0.9247713	total: 2m 17s	remaining: 1m 18s
1271:	learn: 0.9246385	total: 2m 17s	remaining: 1m 18s
1272:	learn: 0.9245763	total: 2m 17s	remaining: 1m 18s
1273:	learn: 0.9245085	total: 2m 17s	remaining: 1m 18s
1274:	learn: 0.9244543	total: 2m 17s	remaining: 1m 18s
1275:	learn: 0.9243418	total: 2m 17s	remaining: 1m 18s
1276:	learn: 0.9242768	total: 2m 18s	remaining: 1m 18s
1277:	learn: 0.9242469	total: 2m 18s	remaining: 1m 18s
1278:	learn: 0.9241799	total: 2m 18s	remaining: 1m 17s
1279:	learn: 0.9241188	total: 2m 18s	remaining: 1m 17s
1280:	learn: 0.9240721	total: 2m 18s	remaining: 1m 17s
1281:	learn: 0.9240454	total: 2m 18s	remaining: 1m 17s
1282:	learn: 0.9239644	total: 2m 18s	remaining: 1m 17s
1283:	learn: 0.9239013	total: 2m 18s	remaining: 1m 17s
1284:	learn: 0.9238156	total: 2m 18s	remaining: 1m 17s
1285:	learn: 0.9237307	total: 2m 18s	remaining: 1m 17s
1286:	learn: 0.9236430	total: 2m 18s	remaining: 1m 17s
1287:	lear

1421:	learn: 0.9141377	total: 2m 33s	remaining: 1m 2s
1422:	learn: 0.9140640	total: 2m 33s	remaining: 1m 2s
1423:	learn: 0.9139797	total: 2m 33s	remaining: 1m 2s
1424:	learn: 0.9139481	total: 2m 33s	remaining: 1m 2s
1425:	learn: 0.9138683	total: 2m 33s	remaining: 1m 1s
1426:	learn: 0.9137930	total: 2m 33s	remaining: 1m 1s
1427:	learn: 0.9137440	total: 2m 34s	remaining: 1m 1s
1428:	learn: 0.9137072	total: 2m 34s	remaining: 1m 1s
1429:	learn: 0.9136240	total: 2m 34s	remaining: 1m 1s
1430:	learn: 0.9134893	total: 2m 34s	remaining: 1m 1s
1431:	learn: 0.9134405	total: 2m 34s	remaining: 1m 1s
1432:	learn: 0.9133160	total: 2m 34s	remaining: 1m 1s
1433:	learn: 0.9132712	total: 2m 34s	remaining: 1m 1s
1434:	learn: 0.9131995	total: 2m 34s	remaining: 1m
1435:	learn: 0.9131530	total: 2m 34s	remaining: 1m
1436:	learn: 0.9130864	total: 2m 35s	remaining: 1m
1437:	learn: 0.9130671	total: 2m 35s	remaining: 1m
1438:	learn: 0.9129854	total: 2m 35s	remaining: 1m
1439:	learn: 0.9129192	total: 2m 35s	remain

1575:	learn: 0.9047671	total: 2m 49s	remaining: 45.7s
1576:	learn: 0.9047002	total: 2m 50s	remaining: 45.6s
1577:	learn: 0.9046267	total: 2m 50s	remaining: 45.5s
1578:	learn: 0.9045375	total: 2m 50s	remaining: 45.4s
1579:	learn: 0.9045280	total: 2m 50s	remaining: 45.3s
1580:	learn: 0.9044616	total: 2m 50s	remaining: 45.2s
1581:	learn: 0.9044411	total: 2m 50s	remaining: 45.1s
1582:	learn: 0.9044091	total: 2m 50s	remaining: 45s
1583:	learn: 0.9043544	total: 2m 50s	remaining: 44.9s
1584:	learn: 0.9043020	total: 2m 50s	remaining: 44.7s
1585:	learn: 0.9041940	total: 2m 51s	remaining: 44.6s
1586:	learn: 0.9041586	total: 2m 51s	remaining: 44.5s
1587:	learn: 0.9040921	total: 2m 51s	remaining: 44.4s
1588:	learn: 0.9040771	total: 2m 51s	remaining: 44.3s
1589:	learn: 0.9040053	total: 2m 51s	remaining: 44.2s
1590:	learn: 0.9039134	total: 2m 51s	remaining: 44.1s
1591:	learn: 0.9038508	total: 2m 51s	remaining: 44s
1592:	learn: 0.9037837	total: 2m 51s	remaining: 43.9s
1593:	learn: 0.9037333	total: 2m

1730:	learn: 0.8957293	total: 3m 6s	remaining: 29s
1731:	learn: 0.8957134	total: 3m 6s	remaining: 28.8s
1732:	learn: 0.8956699	total: 3m 6s	remaining: 28.7s
1733:	learn: 0.8956581	total: 3m 6s	remaining: 28.6s
1734:	learn: 0.8956196	total: 3m 6s	remaining: 28.5s
1735:	learn: 0.8955614	total: 3m 6s	remaining: 28.4s
1736:	learn: 0.8955086	total: 3m 6s	remaining: 28.3s
1737:	learn: 0.8954301	total: 3m 7s	remaining: 28.2s
1738:	learn: 0.8954016	total: 3m 7s	remaining: 28.1s
1739:	learn: 0.8953656	total: 3m 7s	remaining: 28s
1740:	learn: 0.8953274	total: 3m 7s	remaining: 27.9s
1741:	learn: 0.8952483	total: 3m 7s	remaining: 27.8s
1742:	learn: 0.8952112	total: 3m 7s	remaining: 27.7s
1743:	learn: 0.8951711	total: 3m 7s	remaining: 27.6s
1744:	learn: 0.8951279	total: 3m 7s	remaining: 27.4s
1745:	learn: 0.8950564	total: 3m 7s	remaining: 27.3s
1746:	learn: 0.8950410	total: 3m 7s	remaining: 27.2s
1747:	learn: 0.8949937	total: 3m 8s	remaining: 27.1s
1748:	learn: 0.8948807	total: 3m 8s	remaining: 27s

1884:	learn: 0.8881729	total: 3m 23s	remaining: 12.4s
1885:	learn: 0.8880900	total: 3m 23s	remaining: 12.3s
1886:	learn: 0.8880484	total: 3m 23s	remaining: 12.2s
1887:	learn: 0.8879988	total: 3m 23s	remaining: 12.1s
1888:	learn: 0.8879138	total: 3m 23s	remaining: 12s
1889:	learn: 0.8878857	total: 3m 23s	remaining: 11.8s
1890:	learn: 0.8878360	total: 3m 23s	remaining: 11.7s
1891:	learn: 0.8878073	total: 3m 23s	remaining: 11.6s
1892:	learn: 0.8877197	total: 3m 23s	remaining: 11.5s
1893:	learn: 0.8876511	total: 3m 24s	remaining: 11.4s
1894:	learn: 0.8875659	total: 3m 24s	remaining: 11.3s
1895:	learn: 0.8875162	total: 3m 24s	remaining: 11.2s
1896:	learn: 0.8874593	total: 3m 24s	remaining: 11.1s
1897:	learn: 0.8874152	total: 3m 24s	remaining: 11s
1898:	learn: 0.8873878	total: 3m 24s	remaining: 10.9s
1899:	learn: 0.8873490	total: 3m 24s	remaining: 10.8s
1900:	learn: 0.8872323	total: 3m 24s	remaining: 10.7s
1901:	learn: 0.8871852	total: 3m 24s	remaining: 10.6s
1902:	learn: 0.8871529	total: 3m

## Train/test split and attempt purely catboost

In [14]:
X_train, X_test, y_train, y_test = train_test_split(reduced_train.drop('accuracy_group', axis=1)._get_numeric_data(), 
                                                   reduced_train.accuracy_group,
                                                   test_size=.2,
                                                   random_state=42)

In [52]:
from sklearn.metrics import cohen_kappa_score
y_pred = catbooster.predict(X_test)
print(qwk3(y_pred, y_test))  #0.5194
print(cohen_kappa_score(y_pred, y_test, weights='quadratic'))  #0.5194

[0.51940396]
0.5194039619602867


In [21]:
reduced_train.shape, reduced_test.shape

((17690, 891), (1000, 891))

In [23]:
reduced_test

,Unnamed: 0,Clip,Activity,Assessment,Game,acc_Chest Sorter (Assessment),acc_Bird Measurer (Assessment),acc_Cauldron Filler (Assessment),acc_Cart Balancer (Assessment),acc_Mushroom Sorter (Assessment),...,duration_mean,duration_std,accumulated_accuracy,accuracy_group,0,1,2,3,accumulated_accuracy_group,accumulated_actions
0,0,13,7,1,3,-1.0,-1.000000,-1.0,1.0,-1.000000,...,30.000000,0.000000,1.000000,0,0,0,0,1,3.00,866
1,1,27,11,5,12,0.0,0.333333,0.5,1.0,0.500000,...,49.600000,25.757329,0.466667,0,1,1,2,1,1.60,2716
2,2,2,2,0,0,-1.0,-1.000000,-1.0,-1.0,-1.000000,...,0.000000,0.000000,0.000000,0,0,0,0,0,0.00,145
3,3,7,2,0,1,-1.0,-1.000000,-1.0,-1.0,-1.000000,...,0.000000,0.000000,0.000000,0,0,0,0,0,0.00,230
4,4,14,1,0,6,-1.0,-1.000000,-1.0,-1.0,-1.000000,...,0.000000,0.000000,0.000000,0,0,0,0,0,0.00,948
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,3,0,3,1,-1.0,1.000000,-1.0,-1.0,1.000000,...,72.666667,57.978923,0.666667,0,1,0,0,2,2.00,210
996,996,10,2,1,2,-1.0,-1.000000,-1.0,1.0,-1.000000,...,36.000000,0.000000,1.000000,0,0,0,0,1,3.00,301
997,997,31,2,4,0,0.0,-1.000000,1.0,-1.0,0.333333,...,66.000000,48.363209,0.583333,0,1,1,0,2,1.75,524
998,998,10,3,3,1,-1.0,-1.000000,0.0,-1.0,-1.000000,...,26.000000,16.309506,0.250000,0,1,1,1,0,1.00,257


In [27]:
sub_pred = catbooster.predict(reduced_test._get_numeric_data())
len(sub_pred)

1000

## Create submission based on Brad's data prep

In [33]:
sample_sub = pd.read_csv('data/sample_submission.csv')

# To create a submission:
submission = pd.DataFrame()
submission['installation_id'] = sample_sub.installation_id
submission['accuracy_group'] = sub_pred
submission.head()

,installation_id,accuracy_group
0,00abaee7,3.0
1,01242218,3.0
2,017c5718,3.0
3,01a44906,3.0
4,01bc6cb6,3.0


In [35]:
submission.accuracy_group.value_counts(normalize=True)

3.0    0.762
0.0    0.220
1.0    0.018
Name: accuracy_group, dtype: float64

In [36]:
submission.to_csv('preds.csv', index=False)

In [45]:
submission.set_index('installation_id').to_csv('preds16.csv', index=False)

In [49]:
submission.to_csv('preds3.csv', index=False)

In [13]:
quick_eval(reduced_train, rf, cv=True)  #.563cv acc
quick_eval(reduced_train, rc, cv=True)  #.528cv acc
quick_eval(reduced_train, ac, cv=True)  #.529cv acc
quick_eval(reduced_train, et, cv=True)  #.5399cv acc
quick_eval(reduced_train, bc, cv=True)  #.511cv acc
quick_eval(reduced_train, gbc, cv=True)  #.435 cv acc
quick_eval(reduced_train, clf, cv=True) #

# These were not impressive, dropping from future
#quick_eval(reduced_train, lr, cv=True)
#quick_eval(reduced_train, sgd, cv=True)
#quick_eval(reduced_train, nb, cv=True)
#quick_eval(reduced_train, knn, scale=True, cv=True)
#quick_eval(reduced_train, svc, scale=True, cv=True)

The CV score of RandomForestClassifier is 0.565856416054268
The CV score of RidgeClassifier is 0.5280949689089882
The CV score of AdaBoostClassifier is 0.5296212549462973
The CV score of ExtraTreesClassifier is 0.5430186546071226


KeyboardInterrupt: 

## more models, to include sklearn neural net

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier

In [ ]:
mlp = MLPClassifier()
gpc = GaussianProcessClassifier() #takes a long time
#rbf = RBF()
dt = DecisionTreeClassifier()

In [ ]:
#quick_eval(reduced_train, mlp, cv=True)   # 0.295
#quick_eval(reduced_train, gpc) #   .498
#quick_eval(reduced_train, rbf, cv=True)   poor
#quick_eval(reduced_train, dt, cv=True)     #.411

In [ ]:
bc_rf_10 = BaggingClassifier(
    base_estimator=RandomForestClassifier(max_depth=10),
    n_estimators=20)

bc_rf_50 = BaggingClassifier(
    base_estimator=RandomForestClassifier(max_depth=50),
    n_estimators=20)

bc_rf_5 = BaggingClassifier(
    base_estimator=RandomForestClassifier(max_depth=5),
    n_estimators=20)

bc_gbc = BaggingClassifier(
    base_estimator=GradientBoostingClassifier(),
    n_estimators=20)

bc_abc = BaggingClassifier(
    base_estimator=AdaBoostClassifier(),
    n_estimators=20)


quick_eval(reduced_train, bc_rf_5)
quick_eval(reduced_train, bc_rf_10)    # 0.549 with 20 estimators max depth of 10  (.377 with balanced)    0.551 with 200 
quick_eval(reduced_train, bc_rf_50) 
quick_eval(reduced_train, bc_gbc)   # 0.576 with 20 estimators (.389 with balanced)                     0.578 with 200
quick_eval(reduced_train, bc_abc)   # 0.563 with 20 estimators  (.374 with balanced)                    0.561 with 200

The accuracy of BaggingClassifier is 0.5387224420576597
The QWK of BaggingClassifier is 0.18304098870845487
              precision    recall  f1-score   support

           0       0.69      0.20      0.31       826
           1       0.00      0.00      0.00       509
           2       0.00      0.00      0.00       437
           3       0.53      0.98      0.69      1766

    accuracy                           0.54      3538
   macro avg       0.30      0.30      0.25      3538
weighted avg       0.42      0.54      0.42      3538

The accuracy of BaggingClassifier is 0.5729225551158846
The QWK of BaggingClassifier is 0.3474125031192433
              precision    recall  f1-score   support

           0       0.62      0.43      0.51       826
           1       0.15      0.00      0.01       509
           2       0.17      0.00      0.00       437
           3       0.57      0.94      0.71      1766

    accuracy                           0.57      3538
   macro avg       0.38 

## Model Ensembling

In [ ]:
vc = VotingClassifier(estimators=[
    ('Adaboost',ac),
    ('rf', rf), 
    ('gbc', gbc),
    ('et', et),
    #('svc', svc),
    #('gpc', gpc),
    ('rc', rc),
    #('mlp', mlp),
    #('dt', dt),
    ('bc_rf_5', bc_rf_5),
    ('bc_rf_10', bc_rf_10),
    ('bc_rf_50', bc_rf_50),
    ('bc_gbc', bc_gbc),
    ('bc_abc', bc_abc),
    #('catboost', clf)
    ],
     n_jobs=-1,
     voting='hard')

quick_eval(reduced_train, vc)#, cv=True)     #initial w/o catboost .564

In [ ]:
estimators = [
    ('Adaboost',ac),
    ('rf', rf), 
    ('gbc', gbc),
    ('et', et),
    ('gpc', gpc),
    #('svc', svc),
    ('rc', rc),
    #('mlp', mlp),
    #('dt', dt),
    ('bc_rf_5', bc_rf_5),
    ('bc_rf_10', bc_rf_10),
    ('bc_rf_50', bc_rf_50),
    ('bc_gbc', bc_gbc),
    ('bc_abc', bc_abc),
]
stacking_clf = StackingClassifier(estimators=estimators, 
                final_estimator=RandomForestClassifier(), n_jobs=-1)        # created, load from pickle
quick_eval(reduced_train, stacking_clf)

In [ ]:
##joblib.dump(stacking_clf, 'fitted_stacked_classifier.pkl')
##joblib.dump(vc, 'fitted_voting_classifier.pkl')

In [ ]:
test

## Try original datasets

In [ ]:
#trainX = pd.read_csv('trainX.csv')
#test = pd.read_csv('trainY.csv')
#testX = pd.read_csv('testX.csv')

In [ ]:
train = pd.read_csv('full_train.csv')

In [29]:
test = pd.read_csv('original_X_test.csv')

In [ ]:
train = train.drop('game_session', axis=1)
test = test.drop('game_session', axis=1)

In [ ]:
vc = VotingClassifier(estimators=[
    ('Adaboost',ac),
    ('rf', rf), 
    ('gbc', gbc),
    ('et', et),
    #('svc', svc),
    ('rc', rc),
    ('mlp', mlp),
    ('dt', dt),
    ('bc_rf', bc_rf),
    ('bc_gbc', bc_gbc),
    ('bc_abc', bc_abc),
    #('catboost', clf)
    ],
     n_jobs=-1,
     voting='hard')

quick_eval(train, vc)#, cv=True)     #initial w/o catboost .564

In [ ]:
estimators = [
    ('Adaboost',ac),
    ('rf', rf), 
    ('gbc', gbc),
    ('et', et),
    #('svc', svc),
    ('rc', rc),
    ('mlp', mlp),
    ('dt', dt),
    ('bc_rf', bc_rf),
    ('bc_gbc', bc_gbc),
    ('bc_abc', bc_abc),
]
stacking_clf = StackingClassifier(estimators=estimators, 
                final_estimator=RandomForestClassifier(), n_jobs=-1)        # created, load from pickle
quick_eval(train, stacking_clf)

In [ ]:
test.head()

## Predict on original Datasets

In [ ]:
#test_numerics_only = test._get_numeric_data()
final_prediction = vc.predict(test)
sample_sub = pd.read_csv('data/sample_submission.csv')
# To create a submission:
submission = pd.DataFrame()
submission['installation_id'] = sample_sub.installation_id
submission['accuracy_group'] = final_prediction
submission.head()

In [ ]:
submission.accuracy_group.value_counts()

In [ ]:
submission.to_csv('preds2.csv', index=False)
submission.accuracy_group.value_counts()

In [ ]:
from sklearn.metrics import cohen_kappa_score

In [ ]:
cohen_kappa_score(y_pred, y_test)

In [ ]:
reduced_train = pd.read_csv('reduce_train.csv')
reduced_test = pd.read_csv('reduce_test.csv')
reduced_train.shape, reduced_test.shape

In [ ]:
quick_eval(train, rf, cv=True)
quick_eval(reduced_train, rf, cv=True)